# MPC with feed forward Steady-State Target Optimization (SSTO)

The following presents a constrained Linear Quadratic Model Predictive Control (LQ-MPC) with offset-free tracking and disturbance rejection using the feed-forward Steady-State Target Optimization (SSTO) approach for an isolated power system. The MPC control law uses Finite-Receding-Horizon method in dual-mode.



# System and design requirements


<object data="https://drive.google.com/viewerng/viewer?embedded=true&url=https://raw.githubusercontent.com/paulomarconi/MPC-SSTO/master/ACS6116_assignment_only.pdf" width="100%" height="800px"> 
    <p>It appears you don't have a PDF plugin for this browser or JavaScript is disabled. You can <a href="https://drive.google.com/viewerng/viewer?embedded=true&url=https://raw.githubusercontent.com/paulomarconi/MPC-SSTO/master/ACS6116_assignment_only.pdf">download the PDF.</a></p>  
</object>



# The proposed solution

<object data="https://drive.google.com/viewerng/viewer?embedded=true&url=https://raw.githubusercontent.com/paulomarconi/MPC-SSTO/master/report/root.pdf" width="100%" height="800px"> 
    <p>It appears you don't have a PDF plugin for this browser or JavaScript is disabled. You can <a href="https://drive.google.com/viewerng/viewer?embedded=true&url=https://raw.githubusercontent.com/paulomarconi/MPC-SSTO/master/report/root.pdf">download the PDF.</a></p>  
</object>


Source code on [GitHub](https://github.com/paulomarconi/MPC-SSTO).